In [1]:
!gdown --id 1egy4mye5Y_XWDRvQCZ1INYh347hrrdx5
!gdown --id 1zNUnOV_uJPhGLdeqa60GI3MXDe0mjoig
!gdown --id 1t8slD-WEiXlaA3MmTPDFiWvXvRRoIA1Z
!gdown --id 19V6pgcpM9JWNHyhzP0QHupNCRW3s6XH3

Downloading...
From: https://drive.google.com/uc?id=1egy4mye5Y_XWDRvQCZ1INYh347hrrdx5
To: /content/train.csv
14.7MB [00:00, 31.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zNUnOV_uJPhGLdeqa60GI3MXDe0mjoig
To: /content/train_label.csv
100% 9.62k/9.62k [00:00<00:00, 12.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1t8slD-WEiXlaA3MmTPDFiWvXvRRoIA1Z
To: /content/test.csv
3.30MB [00:00, 52.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=19V6pgcpM9JWNHyhzP0QHupNCRW3s6XH3
To: /content/test_nolabel.csv
100% 1.70k/1.70k [00:00<00:00, 2.54MB/s]


In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_train = pd.read_csv('train.csv')
raw_train_label = pd.read_csv('train_label.csv')
raw_test = pd.read_csv('test.csv')
raw_test_nolabel = pd.read_csv('test_nolabel.csv')


# 新增日期欄位
raw_train['arrival_date'] = pd.to_datetime(raw_train[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).apply(' '.join, 1))
raw_train_label[['arrival_date']] = raw_train_label[['arrival_date']].apply(pd.to_datetime)

raw_test['arrival_date'] = pd.to_datetime(raw_test[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).apply(' '.join, 1))
raw_test_nolabel[['arrival_date']] = raw_test_nolabel[['arrival_date']].apply(pd.to_datetime)

In [4]:
raw_train.head()

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,2015-07-01,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,Check-Out,2015-07-03,2015-07-01
2,2,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,Check-Out,2015-07-03,2015-07-01
3,3,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,Check-Out,2015-07-03,2015-07-01
4,4,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,Check-Out,2015-07-03,2015-07-01


# Preprocessing

create training_df and do one-hot encoding

In [5]:
# columns not in test:
print('columns not in test:')
columns_not_in_test = []
for col in raw_train.columns:
    if col not in raw_test.columns:
        columns_not_in_test.append(col)
        print(col)

columns not in test:
is_canceled
adr
reservation_status
reservation_status_date


In [6]:
# 只留下testing data裡有的columns做training
# 合併training和testing一起做one-hot
df_train_test = pd.concat([raw_train.drop(columns=columns_not_in_test, axis=1), raw_test], axis=0)

# Conduct One-hot encoding

In [7]:
# hotel
df_train_test = pd.get_dummies(df_train_test, columns=['hotel'])
# arrival_date_year
df_train_test = pd.get_dummies(df_train_test, 
                               columns=['arrival_date_year', 'arrival_date_month'], 
                               prefix=['arrival_year', 'arrival_month'])

In [8]:
# weekend nights怎麼可以大於2?
# 可以試試這邊不做one-hot的話
df_train_test['stays_in_weekend_nights'].loc[df_train_test['stays_in_weekend_nights'] > 4] = "More than 4"
df_train_test['stays_in_week_nights'].loc[df_train_test['stays_in_week_nights'] > 7] = "More than 7"

df_train_test = pd.get_dummies(df_train_test, 
                               columns=['stays_in_weekend_nights', 'stays_in_week_nights'], 
                               prefix=['weekend_nights', 'week_nights'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [19]:
# 這邊也可以試不做one-hot

df_train_test['adults'].loc[df_train_test['adults'] > 3] = "More than 3"
df_train_test['children'] = df_train_test['children'].fillna(0) # 缺漏值補 0
df_train_test['children'].loc[df_train_test['children'] > 2] = "More than 2"
df_train_test['babies'].loc[df_train_test['babies'] > 0] = "More than 0"

df_train_test = pd.get_dummies(df_train_test, 
                               columns=['adults', 'children', 'babies'], 
                               prefix=['adults', 'children', 'babies'])

# 資料有人數都是0的, 然後有小孩
# mask = (df_train_test['adults'] + df_train_test['children'] + df_train_test['babies']) == 0
# print('# of no people order:', df_train_test[mask].shape)
# # children null補0
# df_train_test['children'].fillna(0, inplace=True)
# # babies沒有null值

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [10]:
df_train_test = pd.get_dummies(df_train_test, 
                               columns=['meal', 'country', 'reserved_room_type', 
                                        'assigned_room_type', 'deposit_type', 'customer_type', 
                                        'market_segment', 'distribution_channel', 'required_car_parking_spaces'])

In [11]:
df_train_test.shape

(119390, 277)

# Validation Set

In [12]:
df_train = df_train_test[:raw_train.shape[0]]
df_test = df_train_test[raw_train.shape[0]:]

not_relevant_columns = ['ID', 'arrival_date_week_number', 'agent', 'company', 'arrival_date']

X_for_train = df_train[df_train['arrival_date'] < '2017-02-01']
y_for_train = np.array(raw_train[raw_train['arrival_date'] < '2017-02-01']['adr'])
X_for_train_stays = np.array(raw_train[raw_train['arrival_date'] < '2017-02-01'])
# X_for_train_stays = np.array(X_for_train['stays_in_weekend_nights'] + X_for_train['stays_in_week_nights'])
X_for_train_arrival = np.array(X_for_train['arrival_date'])

X_for_valid = df_train[df_train['arrival_date'] >= '2017-02-01']
y_for_valid = np.array(raw_train[raw_train['arrival_date'] >= '2017-02-01']['adr'])
X_for_valid_stays = np.array(raw_train[raw_train['arrival_date'] >= '2017-02-01']['stays_in_weekend_nights'] + raw_train[raw_train['arrival_date'] >= '2017-02-01']['stays_in_week_nights'])
# X_for_valid_stays = np.array(X_for_valid['stays_in_weekend_nights'] + X_for_valid['stays_in_week_nights'])
X_for_valid_arrival = np.array(X_for_valid['arrival_date'])

X_for_train = np.array(X_for_train.drop(columns=not_relevant_columns, axis=0))
X_for_valid = np.array(X_for_valid.drop(columns=not_relevant_columns, axis=0))

# Build training model

In [13]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [14]:
import os

import tensorflow as tf
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Sequential
from tensorflow.keras.models import load_model, Model, Sequential

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import ensemble, preprocessing, metrics

In [15]:
def create_NN_model(feat_cnt, layers=2, units=1000):
    feat_input = Input(shape=(feat_cnt,), dtype='float32')
    dense_list = []
    units_cnt = units # 第一層的units數量，每層遞減1/2
    # print(units)
    for i in range(layers):
        if i == 0:
            dense_list.append(Dense(units=units_cnt, activation='relu')(feat_input))
        else:
            dense_list.append(Dense(units=units_cnt, activation='relu')(dense_list[i-1]))
        units_cnt = int(units_cnt/2)

    preds = Dense(units=1)(dense_list[-1])

    # den_1 = Dense(units=100, activation='relu')(feat_input)
    # den_2 = Dense(units=50, activation='relu')(den_1)
    # preds = Dense(units=1)(den_2)

    model = Model(feat_input, preds)
    
    return model

# Tuning

In [33]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
tuning_dict = {}

for layers in range(3, 5):
    for units in range(1500, 2500, 500):
        model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                                layers=layers, 
                                units=units)
        model.compile(loss='mean_squared_error', 
                        optimizer='adam', 
                        metrics=['mean_squared_error'])

        early_stop = EarlyStopping(monitor='val_loss', 
                                    patience=10, 
                                    mode='min')
        
        checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                            monitor='val_loss',
                                            save_weights_only=True, 
                                            mode='min')
        
        history = model.fit(X_for_train, y_for_train, 
                            validation_split=0.1, 
                            callbacks=[early_stop, checkpoint_callback], 
                            epochs=100, 
                            batch_size=10)
        # load_best weights
        trained_model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                                        layers=layers, 
                                        units=units)
        # Load the previously saved weights
        latest = tf.train.latest_checkpoint(checkpoint_dir)
        trained_model.load_weights(latest)

        y_pred_valid = trained_model.predict(X_for_valid)
        y_pred_train = trained_model.predict(X_for_train)

        mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
        mse_train = mean_squared_error(y_for_train, y_pred_train)
        pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}.csv'.format(layers, units))

        # save the trained_model
        trained_model.save('trained_model_NN_layers{}_units{}_valid_mse{}.h5'.format(layers, units, round(mse_valid, 4)))

        print('layers: {}, units: {}'.format(layers, units))
        print('training_mse:', mse_train)
        print('validation_mse:', mse_valid)

Epoch 1/100
7415/7415 [==============================] - 119s 16ms/step - loss: 1486.7777 - mean_squared_error: 1486.7777 - val_loss: 950.5338 - val_mean_squared_error: 950.5338
Epoch 2/100
7415/7415 [==============================] - 115s 15ms/step - loss: 621.5905 - mean_squared_error: 621.5905 - val_loss: 908.9579 - val_mean_squared_error: 908.9579
Epoch 3/100
7415/7415 [==============================] - 115s 15ms/step - loss: 1066.5295 - mean_squared_error: 1066.5295 - val_loss: 864.8408 - val_mean_squared_error: 864.8408
Epoch 4/100
7415/7415 [==============================] - 114s 15ms/step - loss: 681.8542 - mean_squared_error: 681.8542 - val_loss: 791.7057 - val_mean_squared_error: 791.7057
Epoch 5/100
7415/7415 [==============================] - 113s 15ms/step - loss: 473.7560 - mean_squared_error: 473.7560 - val_loss: 889.3026 - val_mean_squared_error: 889.3026
Epoch 6/100
7415/7415 [==============================] - 114s 15ms/step - loss: 436.4072 - mean_squared_error: 436.4

# one-hot後效果更差

In [26]:
# 只留下testing data裡有的columns做training
# 合併training和testing一起做one-hot
df_train_test = pd.concat([raw_train.drop(columns=columns_not_in_test, axis=1), raw_test], axis=0)

In [27]:
# hotel
df_train_test = pd.get_dummies(df_train_test, columns=['hotel'])
# arrival_date_year
df_train_test = pd.get_dummies(df_train_test, 
                               columns=['arrival_date_month'], 
                               prefix=['arrival_month'])

In [ ]:
# weekend nights怎麼可以大於2?
# 可以試試這邊不做one-hot的話
# df_train_test['stays_in_weekend_nights'].loc[df_train_test['stays_in_weekend_nights'] > 4] = "More than 4"
# df_train_test['stays_in_week_nights'].loc[df_train_test['stays_in_week_nights'] > 7] = "More than 7"

# df_train_test = pd.get_dummies(df_train_test, 
#                                columns=['stays_in_weekend_nights', 'stays_in_week_nights'], 
#                                prefix=['weekend_nights', 'week_nights'])

In [28]:
# 這邊也可以試不做one-hot

# df_train_test['adults'].loc[df_train_test['adults'] > 3] = "More than 3"
# df_train_test['children'] = df_train_test['children'].fillna(0) # 缺漏值補 0
# df_train_test['children'].loc[df_train_test['children'] > 2] = "More than 2"
# df_train_test['babies'].loc[df_train_test['babies'] > 0] = "More than 0"

# df_train_test = pd.get_dummies(df_train_test, 
#                                columns=['adults', 'children', 'babies'], 
#                                prefix=['adults', 'children', 'babies'])

# 資料有人數都是0的, 然後有小孩
mask = (df_train_test['adults'] + df_train_test['children'] + df_train_test['babies']) == 0
print('# of no people order:', df_train_test[mask].shape)
# children null補0
df_train_test['children'].fillna(0, inplace=True)
# babies沒有null值

# of no people order: (180, 42)


In [29]:
df_train_test = pd.get_dummies(df_train_test, 
                               columns=['meal', 'country', 'reserved_room_type', 
                                        'assigned_room_type', 'deposit_type', 'customer_type', 
                                        'market_segment', 'distribution_channel', 'required_car_parking_spaces'])

In [48]:
df_train = df_train_test[:raw_train.shape[0]]
df_test = df_train_test[raw_train.shape[0]:]

not_relevant_columns = ['ID', 'arrival_date_week_number', 'agent', 'company', 'arrival_date', 'arrival_date_year']

X_for_train = df_train[df_train['arrival_date'] < '2017-02-01']
y_for_train = np.array(raw_train[raw_train['arrival_date'] < '2017-02-01']['adr'])
X_for_train_stays = np.array(raw_train[raw_train['arrival_date'] < '2017-02-01'])
# X_for_train_stays = np.array(X_for_train['stays_in_weekend_nights'] + X_for_train['stays_in_week_nights'])
X_for_train_arrival = np.array(X_for_train['arrival_date'])

X_for_valid = df_train[df_train['arrival_date'] >= '2017-02-01']
y_for_valid = np.array(raw_train[raw_train['arrival_date'] >= '2017-02-01']['adr'])
X_for_valid_stays = np.array(raw_train[raw_train['arrival_date'] >= '2017-02-01']['stays_in_weekend_nights'] + raw_train[raw_train['arrival_date'] >= '2017-02-01']['stays_in_week_nights'])
# X_for_valid_stays = np.array(X_for_valid['stays_in_weekend_nights'] + X_for_valid['stays_in_week_nights'])
X_for_valid_arrival = np.array(X_for_valid['arrival_date'])



X_for_train = np.array(X_for_train.drop(columns=not_relevant_columns, axis=0))
X_for_valid = np.array(X_for_valid.drop(columns=not_relevant_columns, axis=0))
X_for_test = np.array(df_test.drop(columns=not_relevant_columns, axis=0))

In [49]:
X_for_train.shape[1]

256

In [34]:
# units不用太大 效果不會比較差 跑得又比較快
layers = 2
units = X_for_train.shape[1]

model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                        layers=layers, 
                        units=units)
model.compile(loss='mean_squared_error', 
                optimizer='adam',  # learning rate要調
                metrics=['mean_squared_error'])

early_stop = EarlyStopping(monitor='val_loss', 
                            patience=10, 
                            mode='min')

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                    monitor='val_loss',
                                    save_weights_only=True, 
                                    mode='min')

history = model.fit(X_for_train, y_for_train, 
                    validation_split=0.1, 
                    callbacks=[early_stop, checkpoint_callback], 
                    epochs=100, 
                    batch_size=10)
# load_best weights
trained_model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                                layers=layers, 
                                units=units)
# Load the previously saved weights
latest = tf.train.latest_checkpoint(checkpoint_dir)
trained_model.load_weights(latest)

y_pred_valid = trained_model.predict(X_for_valid)
y_pred_train = trained_model.predict(X_for_train)

mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
mse_train = mean_squared_error(y_for_train, y_pred_train)
pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}.csv'.format(layers, units))

# save the trained_model
trained_model.save('trained_model_NN_layers{}_units{}_valid_mse{}.h5'.format(layers, units, round(mse_valid, 4)))

print('layers: {}, units: {}'.format(layers, units))
print('training_mse:', mse_train)
print('validation_mse:', mse_valid)

Epoch 1/100
7415/7415 [==============================] - 14s 2ms/step - loss: 1895.3182 - mean_squared_error: 1895.3182 - val_loss: 856.9865 - val_mean_squared_error: 856.9865
Epoch 2/100
7415/7415 [==============================] - 14s 2ms/step - loss: 732.3347 - mean_squared_error: 732.3347 - val_loss: 780.7529 - val_mean_squared_error: 780.7529
Epoch 3/100
7415/7415 [==============================] - 14s 2ms/step - loss: 1237.3423 - mean_squared_error: 1237.3423 - val_loss: 758.8596 - val_mean_squared_error: 758.8596
Epoch 4/100
7415/7415 [==============================] - 14s 2ms/step - loss: 1188.3594 - mean_squared_error: 1188.3594 - val_loss: 679.4794 - val_mean_squared_error: 679.4794
Epoch 5/100
7415/7415 [==============================] - 14s 2ms/step - loss: 590.5259 - mean_squared_error: 590.5259 - val_loss: 726.2598 - val_mean_squared_error: 726.2598
Epoch 6/100
7415/7415 [==============================] - 14s 2ms/step - loss: 1433.8938 - mean_squared_error: 1433.8938 - va

In [38]:
# units不用太大 效果不會比較差 跑得又比較快
layers = 2
units = X_for_train.shape[1]*2

model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                        layers=layers, 
                        units=units)
model.compile(loss='mean_squared_error', 
                optimizer='adam',  # learning rate要調
                metrics=['mean_squared_error'])

early_stop = EarlyStopping(monitor='val_loss', 
                            patience=5, 
                            mode='min')

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                    monitor='val_loss',
                                    save_weights_only=True, 
                                    mode='min')

history = model.fit(X_for_train, y_for_train, 
                    validation_split=0.1, 
                    callbacks=[early_stop, checkpoint_callback], 
                    epochs=100, 
                    batch_size=10)
# load_best weights
trained_model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                                layers=layers, 
                                units=units)
# Load the previously saved weights
latest = tf.train.latest_checkpoint(checkpoint_dir)
trained_model.load_weights(latest)

y_pred_valid = trained_model.predict(X_for_valid)
y_pred_train = trained_model.predict(X_for_train)

mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
mse_train = mean_squared_error(y_for_train, y_pred_train)
pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}_valid_mse{}.csv'.format(layers, units, mse_valid))

# save the trained_model
trained_model.save('trained_model_NN_layers{}_units{}_valid_mse{}.h5'.format(layers, units, round(mse_valid, 4)))

print('layers: {}, units: {}'.format(layers, units))
print('training_mse:', mse_train)
print('validation_mse:', mse_valid)

Epoch 1/100
7415/7415 [==============================] - 23s 3ms/step - loss: 2691.6153 - mean_squared_error: 2691.6153 - val_loss: 730.7667 - val_mean_squared_error: 730.7667
Epoch 2/100
7415/7415 [==============================] - 22s 3ms/step - loss: 825.5521 - mean_squared_error: 825.5521 - val_loss: 762.8430 - val_mean_squared_error: 762.8430
Epoch 3/100
7415/7415 [==============================] - 23s 3ms/step - loss: 772.9735 - mean_squared_error: 772.9735 - val_loss: 764.1202 - val_mean_squared_error: 764.1202
Epoch 4/100
7415/7415 [==============================] - 24s 3ms/step - loss: 835.8295 - mean_squared_error: 835.8295 - val_loss: 657.2415 - val_mean_squared_error: 657.2415
Epoch 5/100
7415/7415 [==============================] - 24s 3ms/step - loss: 648.3722 - mean_squared_error: 648.3722 - val_loss: 672.8596 - val_mean_squared_error: 672.8596
Epoch 6/100
7415/7415 [==============================] - 23s 3ms/step - loss: 550.5008 - mean_squared_error: 550.5008 - val_loss

In [39]:
# units不用太大 效果不會比較差 跑得又比較快
layers = 3
units = X_for_train.shape[1]*2

model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                        layers=layers, 
                        units=units)
model.compile(loss='mean_squared_error', 
                optimizer='adam',  # learning rate要調
                metrics=['mean_squared_error'])

early_stop = EarlyStopping(monitor='val_loss', 
                            patience=5, 
                            mode='min')

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                    monitor='val_loss',
                                    save_weights_only=True, 
                                    mode='min')

history = model.fit(X_for_train, y_for_train, 
                    validation_split=0.1, 
                    callbacks=[early_stop, checkpoint_callback], 
                    epochs=100, 
                    batch_size=10)
# load_best weights
trained_model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                                layers=layers, 
                                units=units)
# Load the previously saved weights
latest = tf.train.latest_checkpoint(checkpoint_dir)
trained_model.load_weights(latest)

y_pred_valid = trained_model.predict(X_for_valid)
y_pred_train = trained_model.predict(X_for_train)

mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
mse_train = mean_squared_error(y_for_train, y_pred_train)
pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}_valid_mse{}.csv'.format(layers, units, mse_valid))

# save the trained_model
trained_model.save('trained_model_NN_layers{}_units{}_valid_mse{}.h5'.format(layers, units, round(mse_valid, 4)))

print('layers: {}, units: {}'.format(layers, units))
print('training_mse:', mse_train)
print('validation_mse:', mse_valid)

Epoch 1/100
7415/7415 [==============================] - 26s 4ms/step - loss: 2156.6120 - mean_squared_error: 2156.6120 - val_loss: 799.9923 - val_mean_squared_error: 799.9923
Epoch 2/100
7415/7415 [==============================] - 26s 3ms/step - loss: 1336.5623 - mean_squared_error: 1336.5623 - val_loss: 854.1408 - val_mean_squared_error: 854.1408
Epoch 3/100
7415/7415 [==============================] - 25s 3ms/step - loss: 724.6889 - mean_squared_error: 724.6889 - val_loss: 744.0623 - val_mean_squared_error: 744.0624
Epoch 4/100
7415/7415 [==============================] - 28s 4ms/step - loss: 556.5852 - mean_squared_error: 556.5852 - val_loss: 924.1707 - val_mean_squared_error: 924.1707
Epoch 5/100
7415/7415 [==============================] - 26s 3ms/step - loss: 1043.0041 - mean_squared_error: 1043.0041 - val_loss: 746.5610 - val_mean_squared_error: 746.5610
Epoch 6/100
7415/7415 [==============================] - 26s 4ms/step - loss: 742.0093 - mean_squared_error: 742.0093 - val_

# Fixed Epochs

In [53]:
# units不用太大 效果不會比較差 跑得又比較快
layers = 2
units = X_for_train.shape[1]*2

model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                        layers=layers, 
                        units=units)
model.compile(loss='mean_squared_error', 
                optimizer='adam',  # learning rate要調
                metrics=['mean_squared_error'])

history = model.fit(X_for_train, y_for_train, 
                    validation_split=0.1, 
                    epochs=10, 
                    batch_size=10)

y_pred_valid = model.predict(X_for_valid)
y_pred_train = model.predict(X_for_train)

mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
mse_train = mean_squared_error(y_for_train, y_pred_train)
pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}_valid_mse{}.csv'.format(layers, units, mse_valid))

print('layers: {}, units: {}'.format(layers, units))
print('training_mse:', mse_train)
print('validation_mse:', mse_valid)

Epoch 1/10
7415/7415 [==============================] - 23s 3ms/step - loss: 1894.4064 - mean_squared_error: 1894.4065 - val_loss: 1098.8307 - val_mean_squared_error: 1098.8307
Epoch 2/10
7415/7415 [==============================] - 22s 3ms/step - loss: 1132.8496 - mean_squared_error: 1132.8496 - val_loss: 708.7942 - val_mean_squared_error: 708.7942
Epoch 3/10
7415/7415 [==============================] - 22s 3ms/step - loss: 904.5482 - mean_squared_error: 904.5481 - val_loss: 763.8461 - val_mean_squared_error: 763.8461
Epoch 4/10
7415/7415 [==============================] - 22s 3ms/step - loss: 947.4297 - mean_squared_error: 947.4297 - val_loss: 818.4703 - val_mean_squared_error: 818.4703
Epoch 5/10
7415/7415 [==============================] - 23s 3ms/step - loss: 554.1260 - mean_squared_error: 554.1260 - val_loss: 706.7274 - val_mean_squared_error: 706.7274
Epoch 6/10
7415/7415 [==============================] - 25s 3ms/step - loss: 787.9526 - mean_squared_error: 787.9526 - val_loss: 

In [54]:
y_for_valid.shape

(9147,)

# 訓練True Test

In [45]:
total_X_for_train = np.concatenate((X_for_train, X_for_valid))
total_y_for_train = np.concatenate((y_for_train, y_for_valid))

In [51]:
# units不用太大 效果不會比較差 跑得又比較快
layers = 2
units = X_for_train.shape[1]*2

model = create_NN_model(feat_cnt=X_for_train.shape[1], 
                        layers=layers, 
                        units=units)
model.compile(loss='mean_squared_error', 
                optimizer='adam',  # learning rate要調
                metrics=['mean_squared_error'])

history = model.fit(total_X_for_train, total_y_for_train, 
                    validation_split=0.1, 
                    epochs=10, 
                    batch_size=10)
y_pred = model.predict(X_for_test)

# y_pred_valid = model.predict(X_for_valid)
# y_pred_train = model.predict(X_for_train)

# mse_valid = mean_squared_error(y_for_valid, y_pred_valid)
# mse_train = mean_squared_error(y_for_train, y_pred_train)
# # pd.DataFrame(y_pred_valid, columns=['adr']).to_csv('NN_validPred_layers{}_units{}_valid_mse{}.csv'.format(layers, units, mse_valid))

# print('layers: {}, units: {}'.format(layers, units))
# print('training_mse:', mse_train)
# print('validation_mse:', mse_valid)

Epoch 1/10
8238/8238 [==============================] - 25s 3ms/step - loss: 1500.4314 - mean_squared_error: 1500.4314 - val_loss: 478.7195 - val_mean_squared_error: 478.7195
Epoch 2/10
8238/8238 [==============================] - 24s 3ms/step - loss: 1004.2151 - mean_squared_error: 1004.2151 - val_loss: 430.8516 - val_mean_squared_error: 430.8516
Epoch 3/10
8238/8238 [==============================] - 24s 3ms/step - loss: 610.2182 - mean_squared_error: 610.2182 - val_loss: 389.6386 - val_mean_squared_error: 389.6386
Epoch 4/10
8238/8238 [==============================] - 24s 3ms/step - loss: 707.8491 - mean_squared_error: 707.8491 - val_loss: 455.9375 - val_mean_squared_error: 455.9375
Epoch 5/10
8238/8238 [==============================] - 24s 3ms/step - loss: 1353.4776 - mean_squared_error: 1353.4776 - val_loss: 397.7402 - val_mean_squared_error: 397.7402
Epoch 6/10
8238/8238 [==============================] - 25s 3ms/step - loss: 783.1170 - mean_squared_error: 783.1170 - val_loss: 

In [52]:
pd.DataFrame(y_pred, columns=['adr']).to_csv('NN_testPred_layers{}_units{}'.format(layers, units))